In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Load the Titanic dataset
titanic = sns.load_dataset('titanic')
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

In [4]:
# Check for missing values
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# Age: Fill with median
titanic['age'].fillna(titanic['age'].median(), inplace=True)

In [15]:
# Deck: Fill with 'Unknown'
titanic['deck'] = titanic['deck'].cat.add_categories('Unknown')
titanic['deck'].fillna('Unknown', inplace=True)

In [12]:
titanic['deck'].value_counts()

deck
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: count, dtype: int64

In [16]:
titanic['embarked'].fillna(titanic['embarked'].mode()[0], inplace=True)

In [18]:
# Age: Drop rows with missing values
titanic.dropna(subset=['age'], inplace=True)

In [20]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    2
alive          0
alone          0
dtype: int64

In [21]:
# Convert 'sex' and 'embarked' into numerical values
titanic['sex'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic['embarked'] = titanic['embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# One-hot encoding for 'class' and 'deck'
titanic = pd.get_dummies(titanic, columns=['class', 'deck'], drop_first=True)


In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
titanic[['age', 'fare']] = scaler.fit_transform(titanic[['age', 'fare']])


In [23]:
# Create a new feature 'family_size'
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

# Create a new feature 'is_alone'
titanic['is_alone'] = np.where(titanic['family_size'] == 1, 1, 0)

# Drop irrelevant columns
titanic.drop(columns=['sibsp', 'parch', 'alive'], inplace=True)


In [24]:
# Defining the quantiles
Q1 = titanic[['age', 'fare']].quantile(0.25)
Q3 = titanic[['age', 'fare']].quantile(0.75)
IQR = Q3 - Q1

# Define a function to detect outliers
def detect_outliers(df, Q1, Q3, IQR):
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((df < lower_bound) | (df > upper_bound))

# Apply the function to detect outliers
outliers_age = detect_outliers(titanic['age'], Q1['age'], Q3['age'], IQR['age'])
outliers_fare = detect_outliers(titanic['fare'], Q1['fare'], Q3['fare'], IQR['fare'])

# Handle outliers by capping them to the quantile range
titanic.loc[outliers_age, 'age'] = Q3['age']
titanic.loc[outliers_fare, 'fare'] = Q3['fare']

In [25]:
titanic

survived  pclass  sex       age      fare  embarked    who  adult_male  \
0           0       3    0 -0.565736 -0.502445         0    man        True   
1           1       1    1  0.663861 -0.024246         1  woman       False   
2           1       3    1 -0.258337 -0.488854         0  woman       False   
3           1       1    1  0.433312  0.420730         0  woman       False   
4           0       3    0  0.433312 -0.486337         0    man        True   
..        ...     ...  ...       ...       ...       ...    ...         ...   
886         0       2    0 -0.181487 -0.386671         0    man        True   
887         1       1    1 -0.796286 -0.044381         0  woman       False   
888         0       3    1 -0.104637 -0.176263         0  woman       False   
889         1       1    0 -0.258337 -0.044381         1    man        True   
890         0       3    0  0.202762 -0.492378         2    man        True   

     embark_town  alone  ...  class_Third  deck_B  deck_C  deck_D  deck_E  \
0    Southampton  False  ...         True   False   False   False   False   
1      Cherbourg  False  ...        False   False    True   False   False   
2    Southampton   True  ...         True   False   False   False   False   
3    Southampton  False  ...        False   False    True   False   False   
4    Southampton   True  ...         True   False   False   False   False   
..           ...    ...  ...          ...     ...     ...     ...     ...   
886  Southampton   True  ...        False   False   False   False   False   
887  Southampton   True  ...        False    True   False   False   False   
888  Southampton  False  ...         True   False   False   False   False   
889    Cherbourg   True  ...        False   False    True   False   False   
890   Queenstown   True  ...         True   False   False   False   False   

     deck_F  deck_G  deck_Unknown  family_size  is_alone  
0     False   False          True            2         0  
1     False   False         False            2         0  
2     False   False          True            1         1  
3     False   False         False            2         0  
4     False   False          True            1         1  
..      ...     ...           ...          ...       ...  
886   False   False          True            1         1  
887   False   False         False            1         1  
888   False   False          True            4         0  
889   False   False         False            1         1  
890   False   False          True            1         1  

[891 rows x 21 columns]

In [ ]:
# completed the weak 6 assignment 